# Chapter 15 콘텍스트 관리자와 `else` 블록
- with 문과 콘텍스트 관리자
  - 콘텍스트 관리자 객체의 제어를 받아서 임시로 콘텍스트를 생성하고 신뢰성 있게 제한
  - 에러를 예방하고 반복되는 코드를 줄여줌
  - API를 안전하고 편리하게 사용할 수 있게 만들어줌
- `for`, `while`, `try` 문에서의 `else` 블록

## 15.1 이것 다음에 저것: `if` 문 이외에서의 `else` 블록
`else` 절은 `if` 문뿐만 아니라 `for`, `while`, `try` 문에서도 사용 가능. `if`문에서의 의미와는 다르게 "A를 실행하고 그러고 나서 이것도 실행해라"라는 의미
- `for` 문에서의 `else`: (`break`, `continue`, `return` 등에 의해서가 아니라) `for` 루프가 완전히 실행된 후에 `else` 블록이 실행 
- `while` 문에서의 `else`: (`break`, `continue`, `return` 등에 의해서가 아니라) 조건식이 거짓이 되어 루프를 빠져나온 후에 `else` 블록이 실행
- `try`문에서의 `else`: `try` 블록에서 예외가 발생하지 않았을 때만 `else` 블록 실행
  
~~~python
try:
    dangerous_cell()
    after_cell()
except:
    log('OSError...')
~~~

보다

~~~python
try:
    dangerous_cell()
except:
    log('OSError...')
else:
    after_cell()
~~~

이 에러 발생 여부 확인의 대상 코다가 `dangerous_cell()`임이 명확하다.

In [1]:
my_list = ['apple', 'grape', 'banana']
for item in my_list:
    if item == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')

In [2]:
my_list = ['apple', 'grape', 'banana']
for item in my_list:
    if item == 'strawberry':
        break
else:
    raise ValueError('No strawberry flavor found!')

ValueError: No strawberry flavor found!

## 15.2 콘텍스트 관리자와 `with` 블록
콘텍스트 관리자 객체
- `with` 문을 제어하기 위해 존재
- 프로토콜은 `__enter__()`와 `__exit__()` 메서드로 구성
  - `with` 문이 시작될 때 `__enter__()` 실행되며 `with` 문의 끝에서 `finally` 절의 역할을 수행
  - `with` 문을 빠져나온 후 콘텍스트 관리자 객체의 `__exit__()` 메서드가 호출됌
    - 예외가 발생하지 않으면 `None`, `None`, `None`이 인수로 전달되며,
    - 예외가 발생시 예외 데이터 예외 클래스(`exc_type`), 예외 객체, 예외 메세지 등(`exc_value`), traceback 객체 `traceback`가 인수로 전달된다.

<span style='color: red'>pg555 두 번째 문단. "이 메서드는 `with` 블록의 끝에서 `finally` 절의 역할을 수행한다." 의미</span>

In [3]:
# 콘텍스트 관리자로서 파일 객체의 사용
with open('mirror.py') as fp:
    src = fp.read(50)
    
print(len(src))

print(fp)

print(fp.closed, fp.encoding)

print(fp.read(60))

50
<_io.TextIOWrapper name='mirror.py' mode='r' encoding='UTF-8'>
True UTF-8


ValueError: I/O operation on closed file.

<span style='color: red'>pg555 맨 아래. "콘텍스트 관리자 객체는 `with` 문 뒤의 표현식을 평가한 결과이지만, `as` 절에 있는 타깃 변수의 값은 콘텍스트 관리자 객체의 `__enter__()` 호출 결과" 의미</span>
- 콘텍스트 관리자 객체: open('mirror.py')호출 결과인 `TextIOWrapper` 객체
- 타깃 변수: open('mirror.py') 호출로 나온 객체의 `__enter__()`호출 결과. 이 경우 `self`

In [4]:
from mirror import LookingGlass
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)  # with 문에서는 print() 실행시 sys.stdout.write 대신 LookingGlass.reverse_write()가 실행되기 때문
    
print(what)  # __exit__() 메서드에서 다시 sys.stdout.write을 복구했기 때문

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY


In [5]:
# 예제 15-4 with 블록 없이 LookingGlass 사용하기
from mirror import LookingGlass
manager = LookingGlass()
print(manager)

monster = manager.__enter__()  # __enter__() 메서드가 실행되고, 'JABBERWOCKY'를 monster에 반환
print(monster == 'JABBERWOCKY')
print(monster)
print(manager)

manager.__exit__(None, None, None)  # __exit__() 메서드가 실행됌
print(monster)

eurT
YKCOWREBBAJ
>06a735011x0 ta tcejbo ssalGgnikooL.rorrim<
JABBERWOCKY


## 15.4 표준 라이브러리 `contextlib`의 `@contextmanager` 사용하기
`__enter__()`와 `__exit__()`를 갖는 클래스를 작성하는 대신 `__enter__()` 메서드가 반환할 것을 생성하는 `yield` 문 하나를 가진 제너레이터만 구현하면 된다.
- `yield`가 함수 본체를 두 부분으로 나눠준다.
  - `yield` 문 앞에 있는 모든 코드가 `__enter__()`가 호출될 때 실행
  - `yield` 문 뒤에 있는 모든 코드가 `__exit__()`가 호출될 때 실행

In [6]:
# 예제 15-5 제너레이터로 구현한 콘텍스트 관리자
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])  # 클로저를 통해 접근
    
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'  # with 문의 본체가 실행되는 동안 이 함수는 여기에서 실행을 일시 중단함
    sys.stdout.write = original_write  # with 블록을 빠져나오면 yield 문 이후의 코드가 실행

In [7]:
# 예제 15-6 looking_glass() 콘텍스트 관리자 함수 사용 예
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)
    
print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY


`@contextmanager`의 원리
- `__enter__()`와 `__exit__()`가 정의된 클래스 안에 데코레이트된 함수 (제너레이터)를 넣는다.
  - `__enter__()`에서는 `next(func)`을 통해 `yield`까지 실행
  - `__exit__()`에서는 예외가 전달되었는지 확인하고, 그렇다면 `yield` 행에서 `func.throw(exception)`을 실행. 그렇지 않으면 한 번 더 `next(func)`를 싱행해서 `yield` 문 이후 코드 실행

In [8]:
# 예제 15-7
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)